In [1]:
from tensorflow import keras

/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/y2536zha/anaconda3/envs/w

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from IPython.core.interactiveshell import InteractiveShell
from time import time
from copy import deepcopy
import pandas as pd

import matplotlib as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_rows', 100)

In [3]:
%matplotlib inline 
import sys, os
base_path = os.getcwd()[0:os.getcwd().rfind('Watermark')] + "Watermark/"
sys.path.append(base_path) 

import matplotlib.pyplot as plt
from src.asiaccs_main import asiaccs_blackbox
from src.models import get_deep_cnn_for_cifar, get_lenet_model_for_mnist
from src.preprocess_data import load_cifar_images, load_mnist_images
from src.util import plot_blackbox

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
train, test = load_mnist_images()

In [5]:
x_train, y_train = train
x_test, y_test = test

In [6]:
x_train_original = x_train[:150000]
x_train_surrogate = x_train[150000:]

y_train_original = y_train[:150000]
y_train_surrogate = y_train[150000:]

In [7]:
del x_train
del y_train

In [8]:
x_train_original.shape
y_train_original.shape

x_train_surrogate.shape
y_train_surrogate.shape

x_test.shape
y_test.shape

(150000, 28, 28, 1)

(150000, 10)

(150000, 28, 28, 1)

(150000, 10)

(10000, 28, 28, 1)

(10000, 10)

In [9]:
model_original = get_lenet_model_for_mnist() 

In [10]:
model_original.load_weights('model/best_mnist_model.h5') # original model is trained on x_train_original only

In [11]:
model_original.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2450)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 500)               1225500   
_________________________________________________________________
fc2 (Dense)                  (None, 84)               

In [12]:
import gc
gc.collect()

51

In [13]:
import multiprocessing

multiprocessing.cpu_count()

160

In [14]:
# check model_original performance
model_original.evaluate(x_test, y_test, batch_size = 1000)

10000/10000 [==============================] - 1s 99us/sample - loss: 0.0260 - acc: 0.9924


[0.02602035808376968, 0.9924]

In [15]:
gc.collect()

29

## Implementation for StealML

In [19]:
class TerminateOnBaseline(keras.callbacks.Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='acc', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

def StealMl(model_original, 
            model_surrogate_fun, 
            eval_metric, 
            delta,
            x_train_surrogate, 
            x_test, 
            y_test, 
            path_store_surrogate_model = 'model/best_mnist_model_surrogate.h5',
            batch_size_train = 64,
            batch_size_eval = 1024, # batch size used to evaluate the model
            epochs = 10):
    
    ''' Parameters:
    * model_original: 
        a keras model instance, the model to be stoled  
    * model_surrogate_fun:
        fucntion for creating the surrogate model, need to return a keras model instance 
    * eval_metric:
        evaluation metric used, must be one of the metrics used in model_original
    * delta: 
        Self-explaining
    * x_train_surrogate: 
        Self-explaining
    * x_test: 
        Self-explaining 
    * y_test: 
        Self-explaining
    * path_store_surrogate_model: 
        path used to store the weights of surrogate model,
    * batch_size_train:
        batch_size used to train the model
    * batch_size_eval:
        batch_size used to evaluate the model
    * epochs:
        training epochs):
    '''
    
    # Check Basic setting
    if eval_metric not in model_original.metrics_names:
        raise Exception('Error: eval_metric not in the original model')
        
    eval_metric_index = model_original.metrics_names.index(eval_metric)
    model_original_performance = model_original.evaluate(x_test, y_test, batch_size = batch_size_eval, verbose = 0)[eval_metric_index]
    
    print('Now show basic setting')
    print('original model performance:', eval_metric, model_original_performance)
    print('use', eval_metric, 'as the evaluation metric,', 'delta = ', delta,
      ',stealing will stop as long as the validation', eval_metric, 'achieve', model_original_performance - delta)
    
    
    # Preparation for training
    print('Now prepare for training')
    y_train_surrogate_pred = model_original.predict(x_train_surrogate)
    callbacks = TerminateOnBaseline(monitor = 'val_' + eval_metric, baseline = model_original_performance - delta)
    checkpoint_surrogate = keras.callbacks.ModelCheckpoint( path_store_surrogate_model, verbose=1, monitor='val_acc',save_best_only=True, mode='auto') 
    model_surrogate = model_surrogate_fun()
    
    # train surrogate model
    print('Now start training')
    history = model_surrogate.fit(x_train_surrogate,
                                  y_train_surrogate_pred,
                                  batch_size = batch_size_train, 
                                  epochs = epochs, 
                                  verbose = 1,
                                  validation_data = (x_test, y_test),
                                  callbacks = [checkpoint_surrogate, callbacks])
    
    return (model_surrogate, history)

In [20]:
gc.collect()

28

In [21]:
x_train_surrogate.shape
x_test.shape
y_test.shape

(150000, 28, 28, 1)

(10000, 28, 28, 1)

(10000, 10)

In [31]:
import gc
gc.collect()

117

In [32]:
model_surrogate, history = StealMl( model_original = model_original, 
                                    model_surrogate_fun = get_lenet_model_for_mnist, 
                                    eval_metric = 'acc', 
                                    delta = 0.005,
                                    x_train_surrogate = x_train_surrogate, 
                                    x_test = x_test, 
                                    y_test = y_test, 
                                    path_store_surrogate_model = 'model/best_mnist_model_surrogate.h5',
                                    batch_size_train = 64,
                                    batch_size_eval = 1024, # batch size used to evaluate the model 
                                    epochs = 10)

Now show basic setting
original model performance: acc 0.9924
use acc as the evaluation metric, delta =  0.005 ,stealing will stop as long as the validation acc achieve 0.9873999905586243
Now prepare for training
Now start training
Train on 150000 samples, validate on 10000 samples
Epoch 1/10
149952/150000 [============================>.] - ETA: 0s - loss: 0.0723 - acc: 0.9813
Epoch 00001: val_acc improved from -inf to 0.98050, saving model to model/best_mnist_model_surrogate.h5
150000/150000 [==============================] - 127s 846us/sample - loss: 0.0722 - acc: 0.9813 - val_loss: 0.0652 - val_acc: 0.9805
Epoch 2/10
149952/150000 [============================>.] - ETA: 0s - loss: 0.0354 - acc: 0.9936
Epoch 00002: val_acc did not improve from 0.98050
150000/150000 [==============================] - 127s 845us/sample - loss: 0.0354 - acc: 0.9936 - val_loss: 0.0898 - val_acc: 0.9700
Epoch 3/10
149952/150000 [============================>.] - ETA: 0s - loss: 0.0330 - acc: 0.9947
Epoch 

In [30]:
model_surrogate.evaluate(x_test, y_test)

10000/10000 [==============================] - 7s 681us/sample - loss: 0.0410 - acc: 0.9872


[0.0409906519174343, 0.9872]

In [42]:
history